In [14]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [16]:
trainA = []
trainB = []
for i in range(1,701):
  img = cv2.imread('rain/{}clean.jpg'.format(i))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(256,256))
  trainA.append(img)
  img = cv2.imread('rain/{}bad.jpg'.format(i))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(256,256))
  trainB.append(img)
trainA = np.array(trainA)
trainB = np.array(trainB)
trainA = (trainA - 127.5)/127.5
trainB = (trainB - 127.5)/127.5

In [17]:
# example of defining a u-net encoder-decoder generator model
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.utils.vis_utils import plot_model

# define an encoder block
def define_encoder_block(layer_in, n_filters):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    g = LeakyReLU(alpha=0.2)(layer_in)
    # add downsampling layer
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(g)
    # leaky relu activation
    return g

# define a decoder block
def decoder_block(layer_in, skip_in, n_filters):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    g = Activation('relu')(layer_in)
    # add upsampling layer
    g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(g)
    # merge with skip connection
    g = Concatenate()([g, skip_in])
    return g

# define the standalone generator model
def define_generator(image_shape=(256,256,3)):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# encoder model: C64-C128-C256-C512-C512-C512-C512-C512
	e1 = define_encoder_block(in_image, 64)
	e2 = define_encoder_block(e1, 128)
	e3 = define_encoder_block(e2, 256)
	e4 = define_encoder_block(e3, 512)
	e5 = define_encoder_block(e4, 512)
	e6 = define_encoder_block(e5, 512)
	e7 = define_encoder_block(e6, 512)
	# bottleneck, no batch norm and relu
	b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
	b = Activation('relu')(b)
	# decoder model: C512-C512-C512-C512-C512-C256-C128-C64
	d1 = decoder_block(b, e7, 512)
	d2 = decoder_block(d1, e6, 512)
	d3 = decoder_block(d2, e5, 512)
	d4 = decoder_block(d3, e4, 512)
	d5 = decoder_block(d4, e3, 256)
	d6 = decoder_block(d5, e2, 1280)
	d7 = decoder_block(d6, e1, 64)
	# output
	g = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model

# define image shape
image_shape = (256,256,3)
# create the model
model = define_generator(image_shape)
# summarize the model
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
leaky_re_lu_42 (LeakyReLU)      (None, 256, 256, 3)  0           input_18[0][0]                   
__________________________________________________________________________________________________
conv2d_50 (Conv2D)              (None, 128, 128, 64) 3136        leaky_re_lu_42[0][0]             
__________________________________________________________________________________________________
leaky_re_lu_43 (LeakyReLU)      (None, 128, 128, 64) 0           conv2d_50[0][0]                  
__________________________________________________________________________________________________
conv2d_51 

In [18]:
+from keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Activation, add, Concatenate
from keras.models import Model, model_from_json
from keras.optimizers import Adam
from keras.layers.advanced_activations import ELU, LeakyReLU
from keras.utils.vis_utils import plot_model
from keras.initializers import RandomNormal
from tensorflow.keras.losses import BinaryCrossentropy
def define_discriminator(image_shape):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# source image input
	in_src_image = Input(shape=image_shape)
	# target image input
	in_target_image = Input(shape=image_shape)
	# concatenate images channel-wise
	merged = Concatenate()([in_src_image, in_target_image])
	# C64
	d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
	d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(256, (4,4), padding='same', kernel_initializer=init)(d)
	d = BatchNormalization()(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	patch_out = Activation('sigmoid')(d)
	# define model
	model = Model([in_src_image, in_target_image], patch_out)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss= BinaryCrossentropy(from_logits=True), optimizer=opt, loss_weights=[0.5])
	return model 
# define image shape
image_shape = (256,256,3)
# create the model
model = define_discriminator(image_shape)
# summarize the model
model.summary()
# plot the model
#plot_model(model, to_file='/content/drive/My Drive/test/discriminator_model_plot.png', show_shapes=True, show_layer_names=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
concatenate_34 (Concatenate)    (None, 256, 256, 6)  0           input_19[0][0]                   
                                                                 input_20[0][0]                   
__________________________________________________________________________________________________
conv2d_58 (Conv2D)              (None, 128, 128, 64) 6208        concatenate_34[0][0]             
__________

In [19]:
from tensorflow.keras.losses import BinaryCrossentropy
def define_gan(g_model, d_model1, d_model2, image_shape):
    # make weights in the discriminator not trainable
    d_model1.trainable = False
    d_model2.trainable = False
    # define the source image
    in_src = Input(shape=image_shape)
    in_target = Input(shape = image_shape)
    # connect the source image to the generator input
    gen_out = g_model(in_src)
    # connect the source input and generator output to the discriminator input
    dis_out1 = d_model1([in_src, gen_out])
    dis_out2 = d_model2([in_src, gen_out])
    # src image as input, generated image and classification output
    model = Model([in_src,in_target], [dis_out1, dis_out2, gen_out])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=[BinaryCrossentropy(from_logits=True), BinaryCrossentropy(from_logits=True), 'mae'], optimizer=opt, loss_weights=[1,1,100])
    return model

In [20]:
def generate_real_samples(n_samples, patch_shape):
	# unpack dataset

	# choose random instances
	ix = np.random.randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
	X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels (1)
	y = np.ones((n_samples, patch_shape, patch_shape, 1))
	return [X1, X2], y

In [21]:
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = np.zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

In [22]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, d_model, gan_model, n_samples=1):
	# select a sample of input images
	[X_realA, X_realB], _ = generate_real_samples(n_samples, 1)
	# generate a batch of fake samples
	X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
	# scale all pixels from [-1,1] to [0,1]
	X_realA = (X_realA + 1) / 2.0
	X_realB = (X_realB + 1) / 2.0
	X_fakeB = (X_fakeB + 1) / 2.0
	X_fakeB = 255 * X_fakeB	
	# plot generated target image
	for i in range(n_samples):
		plt.subplot(3, n_samples, 1 + n_samples + i)
		plt.axis('off')
		plt.imshow(X_fakeB[i])
	# save plot to file
	filename1 = 'multi-disc/training/plot_%06d.png' % (step+1)
	cv2.imwrite(filename1,X_fakeB[0])
	# save the generator, discriminator and gan models
	filename2 = 'multi-disc/training/g_model_%06d.h5' % (step+1)
	g_model.save(filename2)
	#filename3 = 'sahil/training/d_model_%06d.h5' % (step+1)
	#d_model.save(filename3)
	#filename4 = 'sahil/training/gan_model_%06d.h5' % (step+1)
	#gan_model.save(filename4)
	print('>Saved: %s, and %s' % (filename1, filename2))

In [23]:
def train(d_model1, d_model2, g_model, gan_model, n_epochs=200, n_batch=1, n_patch=32):
    # unpack dataset
  
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # manually enumerate epochs
    for i in range(n_steps):
        # select a batch of real samples
        [X_realA, X_realB], y_real = generate_real_samples( n_batch, n_patch)
        # generate a batch of fake samples
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realB, n_patch)
        # update discriminator for real samples
        d_loss11 = d_model1.train_on_batch([X_realB, X_realA], y_real)
        # update discriminator for generated samples
        d_loss12 = d_model1.train_on_batch([X_realB, X_fakeB], y_fake)
        # update discriminator for real samples
        d_loss21 = d_model2.train_on_batch([X_realB, X_realA], y_real)
        # update discriminator for generated samples
        d_loss22 = d_model2.train_on_batch([X_realB, X_fakeB], y_fake)
        # update the generator
        g_loss, _, _ = gan_model.train_on_batch([X_realB,X_realA], [y_fake, X_realA])
        # summarize performance
        print('>%d, d11[%.3f] d12[%.3f] d21[%.3f] d22[%.3f] g[%.3f]' % (i+1, d_loss11, d_loss12,d_loss21, d_loss22, g_loss))
    # summarize model performance
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, g_model,d_model1,d_model2, gan_model) 

In [24]:
image_shape = (256,256,3)
# define the models
d_model1 = define_discriminator(image_shape)
d_model2 = define_discriminator(image_shape)
g_model = define_generator(image_shape)
# define the composite model
gan_model = define_gan(g_model, d_model1, d_model2, image_shape)
# train model
train(d_model1, d_model2, g_model, gan_model)

/home/pytorch/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 3 array(s), but instead got the following list of 2 arrays: [array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
    ...